# Radio ML Classification - CNN Approach

Solution to the RadioML classification challenge using CNNs.

In [1]:
# Loading Data

pathToDataset = "C:/Users/aksha/Documents/Jupyter Notebooks/radioml-classification/Datasets/Standard/RML2016.10a_dict.pkl"

# Extract the pickle file
import pickle
import numpy as np
Xd = pickle.load(open(pathToDataset,'rb'),encoding="bytes")
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

# Description of Data

The dataset has the size 220,000×2×128, which means that there are 220,000 entries, each consisting of an array of size 2 × 128. Each array represents the samples of about 128 µs of a received waveform sampled with approximately 106 samples/second, and it contains between 8 and 16 modulation symbols. Since the samples of the signal waveforms are complex-valued, they have been stored as real and imaginary parts, and therefore we have arrays of size 2 × 128 in the data set.

That is, each row is essentially, **a + ib**.

The labels of the downloaded dataset contain two parameters: the modulation technique used (one of [’8PSK’, ’AM-DSB’, ’AM-SSB’, ’BPSK’,’CPFSK’, ’GFSK’, ’PAM4’, ’QAM16’, ’QAM64’, ’QPSK’, ’WBFM’], so 11 possible modulation techniques), and the signal-to noise ratio (SNR) value (one of [−20, −18, −16, −14, −12, −10, −8, −6, −4, −2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18], so 20 possible SNR values). The SNR is a measure for the quality of the communication channel. The higher the SNR, the less “noisy” is the channel.

Each item of the list is essentially like **(b'Modulation Type',SNR Value)** [Use os.fsdecode to extract Modulation Type as string] 

In [2]:
# Import Necessary Packages
%matplotlib inline
import os
import random
import tensorflow.keras.utils
import tensorflow.keras.models as models
from tensorflow.keras.layers import Reshape,Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import *
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras
import numpy as np

# Data Processing

The goal of this stage is to split data into Training, Validation and Testing data and then process the available data and transform (if needed) it into relevant inputs to be fed into the CNN.

1. Split Dataset into Training, Validation and Testing Data : 110,000 samples for training and validation and 110,000 samples for testing.

In [3]:
#  into training and test sets of the form we can train/test on 
np.random.seed(777)

index = np.arange(0,220000)
random.shuffle(index)

trainIdx = index[0:110000]
testIdx = index[110000:220000]

trainX = X[trainIdx]
X_train = np.expand_dims(trainX, axis=-1) # Important

testX = X[testIdx]
X_test = np.expand_dims(testX, axis=-1) # Important

In [4]:
# One Hot Encode Labels
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(np.asarray(lbl)[:,0])
print(lb.classes_)
lbl_encoded=lb.transform(np.asarray(lbl)[:,0])
y_train=lbl_encoded[trainIdx]
y_test=lbl_encoded[testIdx]

[b'8PSK' b'AM-DSB' b'AM-SSB' b'BPSK' b'CPFSK' b'GFSK' b'PAM4' b'QAM16'
 b'QAM64' b'QPSK' b'WBFM']


# Design Neural Network

In [5]:
# Network Parameters
dropoutRate = 0.6

# Structure
inpShape = (2,128,1) # Shape of Input Data
CNN1_numFilt = 256 # Number of Filters in CNN Layer 1
CNN1_kernSize = (1,3) # Kernel Size of CNN Layer 1

CNN2_numFilt = 80 # Number of Filters in CNN Layer 2
CNN2_kernSize = (2,3) # Kernel Size of CNN Layer 2

Dense1_numNeurons = 256 # Number of Nodes in the First Dense Layer
numOutput = 11 # Number of Output Nodes

# Weight Initialization
weightInit = 'glorot_uniform' # Xavier Initialization

# Activation Functions
activationHidden = 'relu'
activationOutput = 'softmax'

# Loss Function
lossFunction = 'categorical_crossentropy'

# Learning Algorithm
netOptimizer = 'adam'

# Construct Network
model = models.Sequential()
model.add(Convolution2D(filters=CNN1_numFilt, kernel_size=CNN1_kernSize, activation=activationHidden, input_shape=inpShape, name='Conv_1'))
model.add(Dropout(dropoutRate))
model.add(Convolution2D(filters=CNN2_numFilt, kernel_size=CNN2_kernSize, activation=activationHidden, name='Conv_2'))
model.add(Dropout(dropoutRate))
model.add(Flatten())
model.add(Dense(Dense1_numNeurons, activation=activationHidden))
model.add(Dropout(dropoutRate))
model.add(Dense(numOutput, activation=activationOutput))
model.compile(loss=lossFunction, optimizer=netOptimizer,metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 2, 126, 256)       1024      
_________________________________________________________________
dropout (Dropout)            (None, 2, 126, 256)       0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 1, 124, 80)        122960    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 124, 80)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9920)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2539776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

# Train Network

In [8]:
history = model.fit(X_train,y_train,epochs=10,batch_size=1024,validation_split=0.1,verbose=2)

Train on 99000 samples, validate on 11000 samples
Epoch 1/10
99000/99000 - 17s - loss: 1.5302 - categorical_accuracy: 0.4216 - val_loss: 1.3933 - val_categorical_accuracy: 0.4718
Epoch 2/10
99000/99000 - 17s - loss: 1.5136 - categorical_accuracy: 0.4266 - val_loss: 1.4080 - val_categorical_accuracy: 0.4673
Epoch 3/10
99000/99000 - 17s - loss: 1.4963 - categorical_accuracy: 0.4325 - val_loss: 1.3693 - val_categorical_accuracy: 0.4802
Epoch 4/10
99000/99000 - 17s - loss: 1.4871 - categorical_accuracy: 0.4364 - val_loss: 1.3746 - val_categorical_accuracy: 0.4756
Epoch 5/10
99000/99000 - 17s - loss: 1.4696 - categorical_accuracy: 0.4405 - val_loss: 1.3627 - val_categorical_accuracy: 0.4797
Epoch 6/10
99000/99000 - 17s - loss: 1.4609 - categorical_accuracy: 0.4425 - val_loss: 1.3454 - val_categorical_accuracy: 0.4891
Epoch 7/10
99000/99000 - 17s - loss: 1.4463 - categorical_accuracy: 0.4476 - val_loss: 1.3412 - val_categorical_accuracy: 0.4768
Epoch 8/10
99000/99000 - 17s - loss: 1.4415 - c

# Test/Evaluate Network

In [7]:
model.evaluate(X_test,y_test)

110000/110000 [==============================] - 24s 218us/sample - loss: 1.4220 - categorical_accuracy: 0.4683


[1.422012652501193, 0.46826363]